In [10]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, find_train_doc_path, test_train_split, clean_1D_arrays, norm_unnorm
from bo_methods_lib.normalize import normalize_p_data, normalize_x, normalize_p_true, normalize_p_bounds
from bo_methods_lib.CS2_create_data import create_sse_data
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from sklearn.preprocessing import MinMaxScaler
import random

# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 300

In [11]:
# 0.61032438 0.43428296 0.72599213 0.66055616 0.52820981 0.95271155
#   0.94534902 0.87020441

In [12]:
X = np.array([[2],[2]])
print(X.shape)

(2, 1)


In [13]:
CS = 1

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])
if CS == 2.2:
    skip_param_types = 1 #This is what changes for subpoint
    Theta_True = Constants[skip_param_types:skip_param_types+2].flatten()
    param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
                  4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
    exp_d = 2
    n = 15 #Number of experimental data points to use
    bounds_p = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                       [ 2,  2,   0,  2,  2,  2, 15,  2]])
    bounds_x = np.array([[-1.5,-0.5],[1,2]])
    LHS = True

else:
    Constants = Theta_True = np.array([1,-1])
    skip_param_types = 0
    param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
    exp_d = 1
    bounds_x = np.array([[-2],[2]])
    bounds_p = np.array([[-2,-2],
                     [2 , 2]])
    LHS = False
    n = 5

print(Constants.shape)
print(Theta_True.shape)

p = 20
d = len(Theta_True)

obj = np.array(["obj"])
emulator = np.array([True])

if emulator == True:
    t = 20*n
else:
    t = 20

# #Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]
Xexp = clean_1D_arrays(Xexp)

m = Xexp.shape[1]
print(Xexp.shape)

theta_mesh = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds_p)
print(theta_mesh.shape)
theta_set = theta_mesh

all_data_doc = find_train_doc_path(emulator, obj, d, t)
all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 
print(all_data.shape)

train_data, test_data = test_train_split(all_data, sep_fact = 0.2, shuffle_seed=9)

if CS == 1 and emulator == False:
    train_p, test_p = train_data[:,1:-2], test_data[:,1:-2]
else:
    train_p, test_p = train_data[:,1:-1], test_data[:,1:-1]
# print(train_p)
print(train_p.shape)
print(test_p.shape)

(2,)
(2,)
(5, 1)
(400, 2)
(100, 5)
torch.Size([20, 3])
torch.Size([80, 3])


In [19]:
#Calculating actual SSE Values
norm = True
bounds_x_scl, scaler_x = normalize_x(bounds_x, None, norm, scaler = None)
bounds_p_scl, scaler_theta = normalize_p_bounds(bounds_p, norm)
Xexp_scl = normalize_x(Xexp, None, norm, scaler = scaler_x)[0]
print(Xexp_scl)
Xexxp = Xexp_scl[0]
print(Xexxp)
print(normalize_x(Xexxp, None, False, scaler_x))

Guess_0 = np.array([0.61032438,0.43428296,0.72599213,0.66055616,0.52820981,0.95271155,0.94534902,0.87020441])
Guess_0 = normalize_p_true(Guess_0, scaler_theta, norm = False)
Guess_1 = np.array([0,1,0.370735521, 9.23236422E-04,0.166088842, 0.169900862,1,0.024381367])
Guess_1 = normalize_p_true(Guess_1, scaler_theta, norm = False)
Guess_2 = Theta_True



SSE_guess_0 = create_sse_data(Guess_0, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
# GP_guess_0 = 1236476096410.422
GP_guess_0 = 74323.09891172957
SSE_guess_1 = create_sse_data(Guess_1, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
GP_guess_1 =-945327999181.6357
SSE_guess_2 = create_sse_data(Guess_2, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
print(Guess_0, Guess_1)
print("True Params \n", Theta_True)
print("True SSE \n", float(SSE_guess_2))
print("Guess 1 \n", np.round(Guess_0,3))
print("True SSE Guess 1 \n", float(SSE_guess_0), "\n GP Guess 1 \n", GP_guess_0, "\n difference %", abs((GP_guess_0 - float(SSE_guess_0))/float(SSE_guess_0)))
# print("Guess 2 \n", np.round(Guess_1,3))
# print("True SSE Guess 2 \n", float(SSE_guess_1), "\n GP Guess 2 \n", GP_guess_1, "\n difference %", abs((GP_guess_1 - float(SSE_guess_1))/float(SSE_guess_1)))
# print(SSE_guess_2, GP_guess_2)

[[0.  ]
 [0.25]
 [0.5 ]
 [0.75]
 [1.  ]]
[0.]
(array([[-2.]]), MinMaxScaler())


ValueError: operands could not be broadcast together with shapes (1,8) (2,) (1,8) 

In [18]:
def norm_unnorm(X, norm = True, scaler = None):
    """
    Normalizes and unnormalizes data
    
    Parameters:
    -----------
        X: ndarray, The array containing the data to be normalized or unnormalized
        norm: bool, default True: Determines whether the data is normalized to 0 and 1 or from 0 and 1 (False)
        scaler: MinMaxScaler(), default None, Must be provided when moving from normalized to unnormalized values
    Returns:
    --------
        X: ndarray, New scaled (norm = True) or unscaled (norm = False) values
        scaler: MinMaxScaler(), The scaler used in the normalization
    """
    
    #Determines whether the original data is a tensor or ndarray
#     print(X[0:3])
    tensor_val = False
    if torch.is_tensor(X) == True:
        tensor_val = True
    #Normalize or unnormalize data
    if norm == True:
        if scaler is None:
            scaler =  MinMaxScaler()
            scaler.fit(X)
#         print("Transforming without scaler")
        X = scaler.transform(X)
    else:
        assert scaler is not None, "Scaler must exist to scale back to normal values"
        X = scaler.inverse_transform(X)
    
    #Make scaled data a tensor if it went into the function as a tensor
    if tensor_val == True and torch.is_tensor(X) == False:
        X = torch.from_numpy(X)
#     print(torch.is_tensor(X)) 
    return X, scaler

def normalize_x(X_val, train_p_x = None, norm = True, scaler = None):
    """
    Normalizes or unnormalizes x data from training/testing data and experimental data
    
    Parameters
    ----------
        X_val: ndarray, experimental x values (Xexp) or Xexp bounds
        train_p_x: ndarray or None, x_values to normalize from train/test data or Xexp when standard approaches are used. Default None
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler: None or MinMaxScaler(), used to un-normalize data or normalize data based on another sets normalization
        
    Returns
    -------
        x_scl: ndarray, rescaled values of x
        scaler_x: MinMaxScaler(), scaler used to obtain these values
    """
    #Change 1D array to 2S with shape (len(X),1)
    X_val = clean_1D_arrays(X_val)
       #Changes train_p values if they exist 
    if train_p_x is not None:
        train_p_x = clean_1D_arrays(train_p_x)
        #If scaling train_p_x data, scale x data using x training data
        x_scl, scaler_x = norm_unnorm(train_p_x, norm, scaler)
        
    else:
        #Scale x data using experimental x data
        x_scl, scaler_x = norm_unnorm(X_val, norm, scaler)
#     print(x_scl)
    return x_scl, scaler_x

def normalize_p_bounds(param_vals_data, norm = True, scaler = None):
    """
    Normalizes or unnormalizes parameter data from training/testing data
    
    Parameters
    ----------
        param_vals_data: ndarray, parameter values to normalize from training/testing data
        m: int, dimensionality of x data
        emulator: bool, whether GP is emulating fxn or error
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler: None or MinMaxScaler(), used to un-normalize data or normalize data based on another sets normalization
        
    Returns
    -------
        param_data_scl: ndarray, rescaled values of x
        scaler_theta: MinMaxScaler(), scaler used to obtain these values
    """
    #Overwite scaled/normal values with normal/scaled values for bounds
    param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm, scaler)
    
    return param_data_scl, scaler_theta

def normalize_p_data(param_vals_data, m, emulator, norm = True, scaler = None):
    """
    Normalizes or unnormalizes parameter data from training/testing data
    
    Parameters
    ----------
        param_vals_data: ndarray, parameter values to normalize from training/testing data
        m: int, dimensionality of x data
        emulator: bool, whether GP is emulating fxn or error
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler: None or MinMaxScaler(), used to un-normalize data or normalize data based on another sets normalization
        
    Returns
    -------
        param_data_scl: ndarray, rescaled values of x
        scaler_theta: MinMaxScaler(), scaler used to obtain these values
    """
    if emulator == True:
        if norm == True:
            #If using emulator approach and normalizing data, overwrite normal parameter values with scaled values
            param_data_scl, scaler_theta = norm_unnorm(param_vals_data[:,0:-m], norm, scaler)
        else:
            #If using emulator approach and un-normalizing data, overwrite scaled values with normal values
            param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm, scaler)
    else:
         #If using standard approach overwrite scaled/normal values with normal/scaled values
        param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm, scaler)
    return param_data_scl

def normalize_p_set(p_set, scaler_theta, norm = True):
    """
    Normalizes or unnormalizes parameter data for theta_set
    
    Parameters
    ----------
        param_vals_data: ndarray, parameter values to normalize from training/testing data
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
           
    Returns
    -------
        p_set_scl: ndarray, rescaled values of x
    """
    #Normalize or unnormalize set data
    p_set_scl, scaler_theta = norm_unnorm(p_set, norm, scaler = scaler_theta)
    return p_set_scl

def normalize_p_true(p_true, scaler_theta, norm= True):
    """
    Normalizes or unnormalizes parameter data for theta_true
    
    Parameters
    ----------
        p_true: ndarray, True parameter values
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
           
    Returns
    -------
        theta_true_scl.flatten(): ndarray, rescaled values of theta_true flattened to correct dimensions
    """    
    #Normalize or un normalize a 2D shape (1, len(theta_true) array and flatten
    theta_true_scl, scaler_theta = norm_unnorm(clean_1D_arrays(p_true, param_clean = True), 
                                               norm, scaler_theta)
    return theta_true_scl.flatten()

def normalize_constants(Constants, p_true, scaler_theta, skip_params, CS, norm = True, scaler_C_before = None, 
                        scaler_C_after = None):
    """
    Normalizes or unnormalizes data for constants
    
    Parameters
    ----------
        Constants: ndarray, ndarray, True values of Muller potential constants OR p_true (CS1 only)
        p_true: ndarray, True parameter values
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        skip_params: int, number of sets of parameters in Constants to skip before reaching the first iterable parameter row
        CS: float, case study label 
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler_C_before: None or MinMaxScaler(), used to un-normalize normalized constansts before theta constants
        scaler_C_after: None or MinMaxScaler(), used to un-normalize normalized constansts after theta constants
        
    Returns
    -------
        x_scl: ndarray, rescaled values of x
        scaler_x: MinMaxScaler(), scaler used to obtain these values
    """   
    #Determine number of parameter types and the length of each: Ex, Muller Case study has 6, and 2D case study has 1
    num_param_types = clean_1D_arrays(Constants).shape[1] #4 - Represents how many indecies are in each param type A, a, b, c, x0, y0
    len_param_type = int(len(p_true)/num_param_types)
    
    #For the case study, the constants are identical to theta_True and we scale them as such
    if CS == 1:
        Constants_scl = normalize_p_true(p_true, scaler_theta, norm)
    #For the Muller potential case studies, we need to normalize constants in chunks
    else:
        #Define constants before, after, and representing theta_true and normalize them by splitting the constants array is 3 parts
#         print(Constants)
        Constants_before, Constants_theta, Constants_after = np.split(Constants, [skip_params,len_param_type+1])
#         print(Constants_before.shape, Constants_theta.shape, Constants_after.shape)
        Constants_before_scl, scaler_C_before = norm_unnorm(Constants_before.T, norm, scaler_C_before)
        
        Constants_theta_scl, scaler_theta =  norm_unnorm(clean_1D_arrays(Constants_theta.flatten(), param_clean = True),
                                                 norm, scaler_theta)
        Constants_theta_scl = Constants_theta_scl.reshape((len_param_type,num_param_types)) #(2,4) for 2.2
        
        Constants_after_scl, scaler_C_after = norm_unnorm(Constants_after.T, norm, scaler_C_after)
        
        #Restack scaled constants and return their value and necessary scalers
        Constants_scl = np.vstack((Constants_before_scl.T, Constants_theta_scl, Constants_after_scl.T))
    return Constants_scl, scaler_C_before, scaler_C_after      

In [19]:
train_p_scl = train_p.clone()
train_p_unscl = train_p.clone()
norm = True
#X values
bounds_x_scl, scaler_x = normalize_x(bounds_x, None, norm, scaler = None)
train_p_scl[:,-m:] = normalize_x(Xexp, train_p[:,-m:], norm, scaler_x)[0]
Xexp_scl = normalize_x(Xexp, None, norm, scaler = scaler_x)[0]
# print(Xexp_scl, train_p_scl[:,-m:], bounds_x_scl)

#Theta Values
bounds_p_scl, scaler_theta = normalize_p_bounds(bounds_p, norm)
train_p_scl[:,0:-m] = normalize_p_data(train_p, m, emulator, norm, scaler_theta)
theta_set_scl = normalize_p_set(theta_set, scaler_theta, norm)
Theta_True_scl = normalize_p_true(Theta_True, scaler_theta, norm)
print(train_p_scl[0:5], train_p[0:5])
# print(bounds_p_scl, train_p_scl[0:5,0:-m], train_p[0:5,0:-m], theta_set_scl[0:5], theta_set[0:5], Theta_True_scl )

train_p_unscl = train_p_scl.clone()
norm = False
#X values
bounds_x_unscl, scaler_x = normalize_x(bounds_x_scl, None, norm, scaler = scaler_x)
print(train_p_scl[0:5])
train_p_unscl[:,-m:] = normalize_x(Xexp, train_p_scl[:,-m:], norm, scaler_x)[0]
print(train_p_scl[0:5])
Xexp_unscl = normalize_x(Xexp, None, norm, scaler = scaler_x)[0]

#Theta values
bounds_p_unscl = normalize_p_bounds(bounds_p, norm, scaler_theta)[0]
# print(train_p_scl[:,0:-m])
train_p_unscl[:,0:-m] = normalize_p_data(train_p_scl[:,0:-m], m, emulator, norm, scaler_theta)
theta_set_unscl = normalize_p_set(theta_set_scl, scaler_theta, norm)
Theta_True_unscl = normalize_p_true(Theta_True_scl, scaler_theta, norm)
print(np.allclose(train_p_unscl, train_p, rtol=1e-10))
print(np.allclose([2,2], [2,2.0000001], rtol=1e-7))
print(train_p_unscl[2:4] , train_p[2:4])

tensor([[0.3342, 0.4147, 0.5350, 0.8630, 0.3860, 0.9109, 0.8506, 0.6507, 0.4701,
         0.7259],
        [0.3342, 0.4147, 0.5350, 0.8630, 0.3860, 0.9109, 0.8506, 0.6507, 0.7684,
         0.1295],
        [0.5134, 0.2081, 0.9344, 0.6391, 0.9617, 0.9742, 0.0992, 0.8902, 0.2140,
         0.9082],
        [0.2065, 0.0857, 0.0198, 0.1111, 0.6642, 0.5042, 0.9570, 0.4541, 0.2720,
         0.4480],
        [0.9702, 0.3203, 0.0866, 0.7915, 0.6117, 0.3164, 0.4600, 0.7390, 0.5130,
         0.5713]], dtype=torch.float64) tensor([[-0.6631, -0.3411, -4.6496,  1.4519, -0.4559,  1.6435, 13.5061,  0.6028,
         -0.3248,  1.3148],
        [-0.6631, -0.3411, -4.6496,  1.4519, -0.4559,  1.6435, 13.5061,  0.6028,
          0.4211, -0.1763],
        [ 0.0537, -1.1677, -0.6558,  0.5563,  1.8467,  1.8968,  5.9925,  1.5609,
         -0.9651,  1.7704],
        [-1.1740, -1.6574, -9.8016, -1.5555,  0.6568,  0.0169, 14.5698, -0.1836,
         -0.8200,  0.6200],
        [ 1.8808, -0.7189, -9.1337,  1.1661,  0

In [6]:
from sklearn.preprocessing import MinMaxScaler
random.seed(10)
X = np.random.randint(-10,10, size = (3,3))
Y = np.random.randint(-10,10, size = (3,3))
print(X)
# print(Y)
# from sklearn.model_selection import LeaveOneOut

# y_true  = []
# y_pred = []

# print(loo.get_n_splits(X))
# for train_index, test_index in loo.split(X):
#     df_test = X[test_index]
#     df_train = X[train_index]
#     print(df_train, df_test)
    

# scaler =  MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)
# print(X)
# X = scaler.inverse_transform(X)
# print(X)

from bo_functions_generic import norm_unnorm
norm = True
X_scl, scaler_X = norm_unnorm(X, norm = norm)
Y_scl, scaler_Y = norm_unnorm(Y, norm = norm)

print(X_scl, scaler_X)
# print(Y, scaler_Y)

X2 = np.array([[0, 1, 0.33],
               [0.25 , 0.75, 0.66]])

X3 = np.array([[2, 5, -2],
               [-5 , 4, 0]])
print(X3)
X3, scaler_X = norm_unnorm(X3, norm = True, scaler = scaler_X)
print(X3)

[[ 2  5  1]
 [-1 -8 -4]
 [ 2  6 -1]]


ModuleNotFoundError: No module named 'bo_functions_generic'

In [ ]:
Xexp_bounds = np.array([[-1.5, -0.5],
                        [1 , 2]])
LHS = gen_theta_set(LHS = False, n_points = 20, dimensions = 2, bounds = Xexp_bounds)
# print(LHS)
# print(LHS.reshape((20,20,-1)).T)


X1 = np.linspace(-1.5,1,20)
X2 = np.linspace(-0.5,2,20)
X_mesh = np.meshgrid(X1,X2)
print(np.array(X_mesh))

In [ ]:
# https://realpython.com/python-timer/
import time
start = 3678393
res = datetime.timedelta(seconds =start)
print(res)

time_tot = 0
x = 0
for i in range(89900):
    start2 = time.time()
    x += 1+2
    end = time.time()

    time_tot+= (end - start2)


print(time_tot)

In [ ]:
A = np.array([[4,3,0,0],
              [4,0,0,3]])
print(A[np.nonzero(A)])

In [ ]:
B = np.zeros((3,3))
B = pd.DataFrame(B)
type(B)
isinstance(B, pd.DataFrame)

In [ ]:
np.random.seed(9)
Array_1 = np.random.rand(3,2)
Array_2 = np.random.rand(3,2)

print(Array_1)
print(Array_2)

ds = gdal.Open('image.tif')
# loop through each band

path = 

with open("converted.raw", "wb") as outfile:
    for bi in range(ds.RasterCount):
        band = ds.GetRasterBand(bi + 1)
        # Read this band into a 2D NumPy array
        ar = band.ReadAsArray()
        print('Band %d has type %s'%(bi + 1, ar.dtype))   
        np.save(outfile, ar.astype('uint16'))